In [ ]:
import pandas as pd
import os
import warnings
warnings.filterwarnings("ignore")
%load_ext autoreload
%autoreload 2

from config.path import Path
data_path = Path

##### Filter data based on IC value size, get selected features names

In [ ]:
""" get selected useful factor name"""
%load_ext autoreload
%autoreload 2

input_para=[1, 1, 'normal_ic', 'All']#
return_time = input_para[0]
factor_back_time = input_para[1]
ic_types=input_para[2]
factor_type=input_para[3]

year = 20
mean_value_method = 'normal_mean'
ic_range=[0.01,1]

file_name = "All_type_%s_%s_tick_level_IC_eachday.csv"%(mean_value_method, year)
ic_result_df = pd.read_csv(data_path+'ic_recal_data/%s'%file_name)

allday_ic_df = pd.read_csv(data_path+'ic_recal_data/'+'All_type_20_tick_level_IC_allday.csv')

select_allday = allday_ic_df[(allday_ic_df['All_day_IC'].abs() >= ic_range[0]) & (allday_ic_df['All_day_IC'].abs() <= ic_range[1])]
allday_sel_cols = list(select_allday['features'])

print("allday IC selected cols num", len(allday_sel_cols))

from ana_code.results_ana import IC_results_ana
ic_object = IC_results_ana(ic_result_df, factor_type, level="IC")
ic_ana_result, selected_cols = ic_object.cal_icstat(ic_range=ic_range)

final_choice_cols = list(set(allday_sel_cols) & set(selected_cols))

print("final choice cols num: ",len(final_choice_cols))

os.chdir(data_path+'/ic_results_data/%s_factor_data/'%factor_type)
fname_list = [x for x in os.listdir() if "2020" in x or "2021" in x]
read_cols = ['Symbol','TradingTime','return'] + final_choice_cols 

print(read_cols)

factor_result_df = pd.DataFrame()
for file in fname_list:
    
    df_oneday = pd.read_hdf(file)
    selected_df = df_oneday[read_cols]
    factor_result_df = pd.concat([factor_result_df,selected_df],ignore_index=True)

print(factor_result_df)

#### calculate the feature through maximum clique algorithm

In [ ]:
%load_ext autoreload
%autoreload 2
from ana_code.results_ana import Factor_results_ana
factor_object = Factor_results_ana(factor_result_df, factor_type, level="Factor")
selected_factor_index = factor_object.select_factor(final_choice_cols,ic_ana_result,corr_threshold=0.5)


#### analysis the distribution and corr between factors using IC value

In [ ]:
from ana_code.results_ana import IC_results_ana
ic_object = IC_results_ana(ic_result_df[selected_factor_index], factor_type, level="IC")
ic_object.corr_heatmap()
ic_object.ic_vs_ic_delta()
ic_object.ic_vs_time()
# ic_object.cal_icstat(ic_file_name="selected")

#### analysis the distribution and corr between factors using factor value

In [ ]:
if 'return' not in selected_factor_index:
    factor_index = selected_factor_index + ['return']
    
factor_object_select = Factor_results_ana(factor_result_df[factor_index+['TradingTime']], factor_type, level="Factor")
factor_object_select.corr_heatmap()
factor_object_select.corr_between_factors_oneday(columns_list = ['C1_aggre_abs_sum','return'])

"""draw all day IC picture"""
factor_object_select.corr_between_factors_allday(columns_list=factor_index)